**Cerinta 1:** Folositi interquartile range pentru a identifica si elimina outlier-ele. O valoare
este considerată outlier dacă este mai mică decât Q1 - 1.5IQR sau mai mare decât Q3 + 1.5IQR,
unde Q1 si Q3 sunt primul si al treilea percentile, iar IQR este diferent, a dintre Q3 si Q1. Dacă
luăm de exemplu variabila "age" din setul de date Titanic, putem calcula Q1 si Q3, apoi IQR.
Valorile care sunt în afara intervalului Q1 - 1.5IQR si Q3 + 1.5IQR pot fi considerate outlier-e
si eliminate.

In [74]:
def iqr(df):
  ############################################
              # Outlieri varsta #
  ############################################

  # calculeaza primul percentile
  ageQ1 = np.percentile(df['Age'], 25)
  # calculeaza al treilea percentile
  ageQ3 = np.percentile(df['Age'], 75)
  # interquartile range
  ageIQR = ageQ3 - ageQ1

  # valoarea sub care se afla outlieri
  ageMinValue = ageQ1 - 1.5*ageIQR
  # valoarea peste care se afla outlieri
  ageMaxValue = ageQ3 + 1.5*ageIQR

  # eliminare outlieri
  df_age_iqr = df[(df['Age'] >= ageMinValue) & (df['Age'] <= ageMaxValue)]

  df_age_iqr.reset_index(drop=True, inplace=True)

  ############################################
              # Outlieri pret #
  ############################################

  # calculeaza primul percentile
  fareQ1 = np.percentile(df['Fare'], 25)
  # calculeaza al treilea percentile
  fareQ3 = np.percentile(df['Fare'], 75)
  # interquartile range
  fareIQR = fareQ3 - fareQ1

  # valoarea sub care se afla outlieri
  fareMinValue = fareQ1 - 1.5*fareIQR
  # valoarea peste care se afla outlieri
  fareMaxValue = fareQ3 + 1.5*fareIQR

  # eliminare outlieri
  df_fare_iqr = df_age_iqr[(df_age_iqr['Fare'] >= fareMinValue) & (df_age_iqr['Fare'] <= fareMaxValue)]

  df_fare_iqr.reset_index(drop=True, inplace=True)

  return df_fare_iqr

**Cerinta 2:** Calculati Z-score pentru fiecare observatie si eliminati valorile care au un Z-score
absolut mai mare decât un anumit prag (de exemplu, 3 sau 4). Z-score reprezintă numărul de
deviatii standard fata de media setului de date. Pentru aceeasi variabilă "age", putem calcula Z-score pentru fiecare vârstă si să eliminăm valorile care au un Z-score mai mare de, să zicem, 3. Valorile cu un Z-score mai mare de 3 ar putea fi considerate outlier-e.

In [75]:
def z_score(df):
  ############################################
              # Outlieri varsta #
  ############################################

  age_zscores = stats.zscore(df['Age'])

  # cel mai asemanator cu iqr este pt 1.9
  df_age_zscore = df[abs(age_zscores) <= 1.9]

  df_age_zscore.reset_index(drop=True, inplace=True)

  ############################################
              # Outlieri pret #
  ############################################

  fare_zscores = stats.zscore(df_age_zscore['Fare'])

  # cel mai asemanator cu iqr este pt 0.7
  df_fare_zscore = df_age_zscore[abs(fare_zscores) <= 0.7]

  df_fare_zscore.reset_index(drop=True, inplace=True)

  return df_fare_zscore

**Cerinta 4:** Dezvoltarea unui model de clasificare pentru prezicerea sanselor de supraviet, uire.

Preprocesarea datelor: încărcarea datelor (pandas), înlăturarea valorilor lipsă (ex: medie),
convertirea coloanele categorice (ex: Sex, Embarked) în valori numerice, normalizarea caracteristicilor numerice.

Protocolul de testare: Împărtiti setul de date în două componente: 80% pentru antrenare
si 20% pentru validare.

Antrenare model: alegerea unui algoritm (Decision-Tree, Random Forest, etc.), antrenare
pe setul de date de antrenament

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#citeste fisiere
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# eliminarea din setul de date de intrare (train si test) a coloanelor 'PassengerId', 'Name', 'Ticket', 'Cabin'
df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis='columns', inplace=True)

#############################################
        #Pregatirea setului de date#
#############################################

#umplerea valorilor goale pt varsta cu media
mean = np.mean(df_train['Age'])
df_train['Age'].fillna(mean, inplace = True)

#umplerea valorilor goale pentru varsta cu cea mai intalnita valoare
mean = df_train['Embarked'].mode()[0]
df_train['Embarked'].fillna(mean, inplace=True)

#eliminare outlieri dupa varsta si pret
df_final = z_score(df_train)

#scrierea in "filtered_train.csv" a datelor filtrate
df_final.to_csv('filtered_train.csv')

# transforma Sex si Embarked in coloane numerice
le = LabelEncoder()
le.fit(df_final['Sex'])
df_final.loc[:, 'Sex'] = le.transform(df_final['Sex'])

le = LabelEncoder()
le.fit(df_final['Embarked'])
df_final.loc[:, 'Embarked'] = le.transform(df_final['Embarked'])

#############################################
 #Impartirea setului de date pentru testare#
#############################################

# setul de date de intrare (train si test)
X = df_final.drop(columns=['Survived'])

# normalizarea valorilor care nu sunt coloane categoriale
# scot sex si embarked
sex = X['Sex']
embarked = X['Embarked']
X.drop(['Sex', 'Embarked'], axis='columns', inplace=True)

scaler = StandardScaler()
X_mat = scaler.fit_transform(X)
X = pd.DataFrame(X_mat, columns=X.columns)

#adaug sex si embarked
X['Sex'] = sex
X['Embarked'] = embarked

# setul de date de iesire (train si test) contine survived
y = df_final['Survived']

# imparte setul de date
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train = X
y_train = y

#pregatire date de testare
mean = np.mean(df_test['Age'])
df_test['Age'].fillna(mean, inplace = True)

mean = np.mean(df_test['Fare'])
df_test['Fare'].fillna(mean, inplace = True)

id = df_test['PassengerId']
sex = df_test['Sex']
embarked = df_test['Embarked']

df_test.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Sex', 'Embarked'], axis='columns', inplace=True)
scaler = StandardScaler()
X_mat = scaler.fit_transform(df_test)
df_test = pd.DataFrame(X_mat, columns=df_test.columns)

df_test['Sex'] = sex
df_test['Embarked'] = embarked

le = LabelEncoder()
le.fit(df_test['Sex'])
df_test.loc[:, 'Sex'] = le.transform(df_test['Sex'])

le = LabelEncoder()
le.fit(df_test['Embarked'])
df_test.loc[:, 'Embarked'] = le.transform(df_test['Embarked'])

X_test = df_test

# antrenare cu decision tree
rfc = RandomForestClassifier(random_state=14)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)
df_out = pd.DataFrame({
    'PassengerId': id,
    'Survived': y_pred
})
df_out.to_csv('prediction.csv', index=False)

#accuracy = accuracy_score(y_test, y_pred)
#print(accuracy)